****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import json

 ### Load the dataset
 
 The json file is located in 'data/world_bank_projects.json'

In [2]:
with open('data/world_bank_projects.json','r') as json_file:
    json_data = json.load(json_file)


### Turn the json file into a pandas DataFrame

In [3]:
df = pd.DataFrame(json_data)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null object
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 no

## Exercise 1

Observe that the projects may be assigned to whole continents/regions. We filter these projects out so as to maintain the projects relative to single countries only. The projects to be filtered out have the entry in the column countrycode equal to a number and a capital letter instead of two capital letters. 

In [5]:
import re

# Pattern identifies the string associated to a proper country code
pattern = re.compile('[A-Z_][A-Z_]')
non_continents = [bool(pattern.match(x)) for x in df['countrycode']]

# We keep only the entries associated to single country projects
df_non_continents = df[non_continents]

In [6]:
NumProjects = df_non_continents['countryname'].value_counts()
NumProjects[:10]

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Islamic Republic of Pakistan        9
Name: countryname, dtype: int64

## Exercises 2 and 3

The various projects have multiple namecodes, the entry of the column 'mjtheme_namecode' is a list of dictionaries (see the cell below). Each dictionary has a key 'code' that contains the code of the project and a key 'name' that refers to the code and is missing. 

In [7]:
df.loc[0,'mjtheme_namecode']   # Note the missing name in the second dictionary

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

Ex3. We now create a dictionary that matches the code (used as key) to the relative name. Then we use the dictionary to fill the blank names in the column 'mjtheme_namecode'

In [8]:
MyDict = {}

# We read the column 'mjtheme_namecode' and add the codes:names to MyDict 
for Project_Code_List_Dict in df['mjtheme_namecode'] :
    for Project_Code_Dict in Project_Code_List_Dict :
        # We enter the if clause if the code is missing in MyDict and the name associated is present 
        if (Project_Code_Dict['code'] not in MyDict) & (Project_Code_Dict['name'] != '') :
            MyDict[Project_Code_Dict['code']] = Project_Code_Dict['name']

# We now have MyDict at disposal, i.e., codes as keys and names as values
# We read again the column 'mjtheme_namecode' and add the names where needed
for num_row, Project_Code_List_Dict in enumerate(df['mjtheme_namecode']) :
    for num_dict, Project_Code_Dict in enumerate(Project_Code_List_Dict) :
        # We enter the if clause if the name is missing
        if (Project_Code_Dict['name'] == '') :
            # Read the name from MyDict
            NameToAdd = MyDict[Project_Code_Dict['code']]
            # Access the correct point in the DataFrame
            df.loc[num_row,'mjtheme_namecode'][num_dict]['name'] = NameToAdd

df.loc[0,'mjtheme_namecode']       # Note that the missing name in the second dictionary is now present     

[{'code': '8', 'name': 'Human development'},
 {'code': '11', 'name': 'Environment and natural resources management'}]

Ex2. We now write a function extracting the list of codes from a list of dictionaries with a code entry.

In [9]:
def Extract_Proj_Codes(List_Dict):
    List_Values = [d['code']  for d in List_Dict]
    return List_Values

In [10]:
# Use list comprehension and Extract_Proj_Codes to obtain a list of lists containing the codes of the projects
List_Codes_Matrix = [Extract_Proj_Codes(ProjCodes) for ProjCodes in df['mjtheme_namecode']]

# Flatten the list of lists and change the codes in their names through MyDict
List_Codes = [MyDict[code] for sublist in List_Codes_Matrix for code in sublist]

In [11]:
from collections import Counter

# Print the 10 most common type of projects with their number
Counter(List_Codes).most_common(10)

[('Environment and natural resources management', 250),
 ('Rural development', 216),
 ('Human development', 210),
 ('Public sector governance', 199),
 ('Social protection and risk management', 168),
 ('Financial and private sector development', 146),
 ('Social dev/gender/inclusion', 130),
 ('Trade and integration', 77),
 ('Urban development', 50),
 ('Economic management', 38)]